In [1]:
%run "Funciones_Prepara_Prediccion.ipynb"
%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

import pandas as pd
import numpy as np
import joblib
import time

In [2]:
# Datos base sólo para obtener parámetros: TEMPORADA, DESC_TIEMPO y ESTACIONES
datos = pd.read_csv("../../Data/DataFrame_Final_Cierre_Cluster.csv",parse_dates=['FECHA'])

In [3]:
db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 
                          'Meteo_US_NivelHora')
data_Meteo = _baseMeteorologia(db_Meteo)

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')


In [4]:
data_festivos = _baseFestivos()

In [5]:
df_estaciones = datos.ESTACION.unique()
# df_estaciones = df_estaciones[:3]
df_estaciones

array([  6,   8,   9,  12,  15,  17,  19,  20,  21,  25,  26,  31,  36,
        39,  40,  41,  43,  44,  45,  47,  48,  49,  50,  52,  53,  54,
        55,  56,  57,  58,  59,  63,  65,  66,  67,  69,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  85,  87,  90,  92,  93,
        95,  96,  98, 101, 102, 103, 104, 105, 106, 107, 110, 113, 114,
       122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 139, 145, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158,
       160, 161, 162, 163, 164, 166, 167, 168, 170, 172, 175,   7,  11,
        13,  14,  16,  18,  27,  34,  51,  64,  71,  83,  91,  94, 118,
       140, 141, 142, 143, 147,   5,  42,  46,  62,  86, 117, 121, 137,
       165,   4, 111, 116, 146, 151,  30,  89, 108, 112, 159, 169,   3,
        60,  88,  97, 109, 115, 120, 138, 171,  33,  84,  99, 127, 144,
       173, 174,   1, 100,  10,  35,  38,   2,  32,  37, 119,  61,  29,
        28], dtype=int64)

In [6]:
# Generación de datos para Dummies

dataDummy = _agregaRegistrosParaDummies(datos.TEMPORADA.unique(), datos.DESC_TIEMPO.unique())

cat_cols= ['MES', 'DIA_SEMANA', 'TEMPORADA', 'Es_Festivo', 'Es_FinSemana','DESC_TIEMPO']
num_cols= ['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION']

In [7]:
fecha_inicio = pd.to_datetime('01/09/2020', format='%d/%m/%Y')
fecha_termino = fecha_inicio + pd.DateOffset(29)

# predsDiarias = pd.DataFrame(columns=['ESTACION', 'FECHA', 'PREDICCION'])

t_ini = time.time()

for estacion in df_estaciones:
    
    print(estacion)

    model = joblib.load('../Modelos/Modelo_'+ str(estacion) +'.pkl')
    
    for fecha in pd.date_range(start=fecha_inicio, end=fecha_termino):
        FechaAPredecir = _estacionDataFechaNueva(estacion, fecha, data_Meteo, data_festivos, dataDummy, cat_cols, num_cols)

        # Prediccion según modelo(estacion)
        prediccion = round(model.predict(FechaAPredecir)[0])
        prediccion = pd.DataFrame([str(estacion), fecha.year, fecha.month, fecha.day, prediccion]).T
        
        # Se crea predsFechas sólo en primera ejecución de los loops
        if ((fecha==fecha_inicio) & (estacion==df_estaciones[0])):
            print('ini')
            Fechas = FechaAPredecir.copy()
            Predicciones = prediccion.copy()
        else:
            Fechas = pd.concat([Fechas, FechaAPredecir])
            Predicciones = pd.concat([Predicciones, prediccion])
        
t_end = time.time()
print ((t_end - t_ini)/60)

Predicciones.rename(columns={0:'ESTACION', 1:'ANIO', 2:'MES', 3:'DIA', 4:'PREDICCION'}, inplace=True)
Predicciones['ESTACION']=Predicciones['ESTACION'].astype(int)
Predicciones['ANIO']=Predicciones['ANIO'].astype(int)
Predicciones['MES']=Predicciones['MES'].astype(int)
Predicciones['DIA']=Predicciones['DIA'].astype(int)
Predicciones['PREDICCION']=Predicciones['PREDICCION'].astype(int)


6


C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed o

ini


C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed o

8


...

C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed o

28


C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed o

1.576863944530487


C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\gonza\AppData\Roaming\Python\Python39\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [8]:
Predicciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ESTACION    5100 non-null   object
 1   ANIO        5100 non-null   object
 2   MES         5100 non-null   object
 3   DIA         5100 non-null   object
 4   PREDICCION  5100 non-null   int32 
dtypes: int32(1), object(4)
memory usage: 219.1+ KB


In [31]:
Predicciones

,ESTACION,ANIO,MES,DIA,PREDICCION
0,6,2020,9,1,96
0,6,2020,9,2,96
0,6,2020,9,3,100
0,6,2020,9,4,92
0,6,2020,9,5,81
...,...,...,...,...,...
0,28,2020,9,26,15
0,28,2020,9,27,16
0,28,2020,9,28,16
0,28,2020,9,29,16


In [10]:
Predicciones.to_csv('../../data/Predicciones.csv', index=False)

In [11]:
db_Tracks = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 
                           'Tracks_Demanda')

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Tracks_Demanda')


In [12]:
dataReal = _data_anio_mes(db_Tracks, 2020, 9)

In [13]:
dataReal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432520 entries, 0 to 432519
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ANIO              432520 non-null  int64         
 1   MES               432520 non-null  int64         
 2   DIA               432520 non-null  int64         
 3   HORA              432520 non-null  int64         
 4   DIA_SEMANA        432520 non-null  int64         
 5   AM_PM             432520 non-null  object        
 6   TEMPORADA         432520 non-null  object        
 7   TEMPORADA_NUM     432520 non-null  int64         
 8   UNPLUG_TIME_date  432520 non-null  datetime64[ns]
 9   travel_time       432520 non-null  int64         
 10  idunplug_station  432520 non-null  int64         
 11  user_type         432520 non-null  int64         
 12  ageRange          432520 non-null  int64         
 13  zip_code          432520 non-null  object        
 14  Es_F

In [19]:
dataRealEstacion = dataReal.groupby(['idunplug_station', 'ANIO', 'MES', 'DIA']).agg(DEMANDA=('DEMANDA','sum'))
dataRealEstacion.reset_index(inplace=True)
dataRealEstacion.rename(columns={'idunplug_station':'ESTACION'}, inplace=True)
dataRealEstacion

,ESTACION,ANIO,MES,DIA,DEMANDA
0,1,2020,9,1,56
1,1,2020,9,2,48
2,1,2020,9,3,45
3,1,2020,9,4,64
4,1,2020,9,5,60
...,...,...,...,...,...
6386,221,2020,9,28,41
6387,221,2020,9,29,75
6388,221,2020,9,30,63
6389,2008,2020,9,2,1


In [24]:
tmpEstaciones=pd.DataFrame(df_estaciones)
tmpEstaciones.rename(columns={0:'ESTACION'},inplace=True)
dataRealEstacion = pd.merge(tmpEstaciones, dataRealEstacion)

In [28]:
dataRealEstacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5033 entries, 0 to 5032
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ESTACION  5033 non-null   int64
 1   ANIO      5033 non-null   int64
 2   MES       5033 non-null   int64
 3   DIA       5033 non-null   int64
 4   DEMANDA   5033 non-null   int64
dtypes: int64(5)
memory usage: 235.9 KB


In [32]:
Predicciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   ESTACION    5100 non-null   int32
 1   ANIO        5100 non-null   int32
 2   MES         5100 non-null   int32
 3   DIA         5100 non-null   int32
 4   PREDICCION  5100 non-null   int32
dtypes: int32(5)
memory usage: 139.5 KB


In [33]:
dataComparada=pd.merge(Predicciones, dataRealEstacion, how='left', left_on=['ESTACION','ANIO','MES','DIA'],
                      right_on=['ESTACION','ANIO','MES','DIA'])
dataComparada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 5099
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ESTACION    5100 non-null   int32  
 1   ANIO        5100 non-null   int32  
 2   MES         5100 non-null   int32  
 3   DIA         5100 non-null   int32  
 4   PREDICCION  5100 non-null   int32  
 5   DEMANDA     5033 non-null   float64
dtypes: float64(1), int32(5)
memory usage: 179.3 KB


In [34]:
dataComparada

,ESTACION,ANIO,MES,DIA,PREDICCION,DEMANDA
0,6,2020,9,1,96,116.0
1,6,2020,9,2,96,109.0
2,6,2020,9,3,100,109.0
3,6,2020,9,4,92,99.0
4,6,2020,9,5,81,94.0
...,...,...,...,...,...,...
5095,28,2020,9,26,15,7.0
5096,28,2020,9,27,16,15.0
5097,28,2020,9,28,16,12.0
5098,28,2020,9,29,16,13.0


In [35]:
dataComparada.to_csv('../../data/PrediccionComparadaReal.csv', index=False)